In [1]:
from IPython import get_ipython


def type_of_script():
    """
    Detects and returns the type of python kernel
    :return: string 'jupyter' or 'ipython' or 'terminal'
    """
    try:
        ipy_str = str(type(get_ipython()))
        if 'zmqshell' in ipy_str:
            return 'jupyter'
        if 'terminal' in ipy_str:
            return 'ipython'
    except:
        return 'terminal'


if type_of_script() == 'jupyter':
    from tqdm.notebook import tqdm
else:
    from tqdm import tqdm
    
import matplotlib.pyplot as plt # type: module
import matplotlib.ticker as ticker
from matplotlib import colormaps
from matplotlib.colors import Normalize

import numpy as np
import os, glob
import time
import warnings

from rur.fortranfile import FortranFile
from rur import uri, uhmi, painter, drawer
from rur.sci.photometry import measure_luminosity
from rur.sci.geometry import get_angles, euler_angle
from rur.utool import rotate_data
from scipy.ndimage import gaussian_filter
uri.timer.verbose=1
# from rur.sci.kinematics import f_getpot

from icl_IO import mode2repo, pklsave, pklload
from icl_tool import *
from icl_numba import large_isin, large_isind, isin
from icl_draw import drawsnap, add_scalebar, addtext, MakeSub_nolabel, label_to_in, fancy_axis, circle
import argparse, subprocess
from importlib import reload
import cmasher as cmr
from copy import deepcopy
from multiprocessing import Pool, shared_memory

In [4]:
mode = 'nh'
iout = 1026
repo, rurmode, dp = mode2repo(mode)
snap = uri.RamsesSnapshot(repo, iout, mode=rurmode)
snaps = uri.TimeSeries(snap)
snaps.read_iout_avail()
nout = snaps.iout_avail['iout']
gals = uhmi.HaloMaker.load(snap, galaxy=True, double_precision=dp)
hals = uhmi.HaloMaker.load(snap, galaxy=False, double_precision=dp)
database = f"/home/jeon/MissingSat/database/nh"

from common_func import *

[Output 01026] Age (Gyr) : 11.624 / 13.741, z = 0.17149 (a = 0.8536)


In [39]:
tmp = pklload(f"{database}/main_prog/mainhalos_0.pickle")

In [40]:
tmp.keys()

dict_keys([1, 3, 7, 8, 10, 23, 25, 26, 36, 64, 121, 129, 11])

In [41]:
tmp[1]['timestep']

array([1020, 1010, 1000,  990,  980,  970,  960,  950,  940,  930,  920,
        910,  900,  890,  880,  870,  860,  850,  840,  830,  820,  810,
        800,  790,  780,  770,  760,  740,  730,  720,  710,  690,  660,
        650,  640,  620,  610,  600,  590,  580,  550,  520,  510,  500,
        490,  480,  450,  430,  420,  410,  400,  390,  380,  370,  360,
        350,  340,  330,  320,  310,  300,  260,  230,  220,  210,  200,
        180,  170,  160,  150,  140,  120,  110,  100,   90,   80,   70,
         60,   50,   40], dtype=int32)

In [42]:
tmp[1]['timestep']

array([1020, 1010, 1000,  990,  980,  970,  960,  950,  940,  930,  920,
        910,  900,  890,  880,  870,  860,  850,  840,  830,  820,  810,
        800,  790,  780,  770,  760,  740,  730,  720,  710,  690,  660,
        650,  640,  620,  610,  600,  590,  580,  550,  520,  510,  500,
        490,  480,  450,  430,  420,  410,  400,  390,  380,  370,  360,
        350,  340,  330,  320,  310,  300,  260,  230,  220,  210,  200,
        180,  170,  160,  150,  140,  120,  110,  100,   90,   80,   70,
         60,   50,   40], dtype=int32)

In [43]:
keys = list(tmp.keys())
for key in keys:
    print( tmp[key]['timestep'] )

[1020 1010 1000  990  980  970  960  950  940  930  920  910  900  890
  880  870  860  850  840  830  820  810  800  790  780  770  760  740
  730  720  710  690  660  650  640  620  610  600  590  580  550  520
  510  500  490  480  450  430  420  410  400  390  380  370  360  350
  340  330  320  310  300  260  230  220  210  200  180  170  160  150
  140  120  110  100   90   80   70   60   50   40]
[1020 1010 1000  990  980  970  960  950  940  930  920  910  900  890
  880  870  860  850  840  830  820  810  800  790  780  770  760  740
  730  720  710  690  660  650  640  620  610  600  590  580  550  520
  510  500  490  480  450  430  420  410  400  390  380  370  360  350
  340  330  320  310  300  260  230  220  210  200  180  170  160  150
  140  120  110  100   90   80   70   60]
[1020 1010 1000  990  980  970  960  950  940  930  920  910  900  890
  880  870  860  850  840  830  820  810  800  790  780  770  760  740
  730  720  710  690  660  650  640  620  610  600  59

In [19]:
tmp.keys()

dict_keys([1, 3, 7, 8, 10, 23, 25, 26, 36, 64, 121, 129, 11])

In [30]:
tmp[1]['r200_code'] < tmp[1]['rvir']

array([False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False])

In [31]:
for key in keys:
    print(np.isnan(tmp[key]['fcontam_200']))

[False False False False False False False False False False False False
 False False False False False False False False False False False False
 False False False False False False False False False False False False
 False False False False False False False False False False False False
 False False False False False False False False False False False False
 False False False False False False False False False False False False
 False False False False False False False  True]
[False False False False False False False False False False False False
 False False False False False False False False False False False False
 False False False False False False False False False False False False
 False False False False False False False False False False False False
 False False False False False False False False False False False False
 False False False False False False False False False False False False
 False False False False False  True]
[False False False False False False

In [44]:
tmp[1]

array([(6451483,  1, 1020, 1,  1, 0,  56, 1120, 0.85097384, 5.70013954e+10, 0.46675178, 0.47556914, 0.51980202, 37.82228584,  59.11487408,  40.67454944,  4.35816889e+09,  1.94866159e+10, -3.41013287e+10, 6.02635932e-04, 0.00977035, 0.00792019, 0.00464194, 1.83447274e+04, -5.67704119e+03, 1.26676862e+04, 0.42155428, 194.02335876, 194.73591172, 2.41830373e-01, 5.82998387e-04, 5.70013954e+10, 124893.11508485, 58.98234233,   129.61130847, 5.82998387e-04, 1311, 941959, 1, 1311, 0, 0.46675177, 0.47556913, 0.51980153, 47.08042025, 60.19539841,  45.48113543, 1.17818074e+12, 0.00204155, 8.49287554e-06, 4.85891779e-07, False, False, 171.23596669, 9.38955858e+11, 0.00141661, 6.04195322e+10, 4.96309962e+10,  1., 0., 0., 0., 0, 0, 0.,  1,  1, 0.49344598, 0.49278612),
       (6434521,  1, 1010, 1,  1, 0,  52, 1123, 0.84639794, 5.68516817e+10, 0.46673033, 0.4755344 , 0.51977854, 38.0433488 ,  59.99293362,  40.28748667,  4.39606289e+09,  1.91923100e+10, -3.40665837e+10, 5.78277647e-04, 0.00979708, 0.0

In [45]:
LG = pklload(f"{database}/LG")
gtree = pklload("/storage6/NewHorizon/ptree/ptree_stable.pkl")
rgtree = {}
for key in tqdm( keys ):
    if(LG[key]['isLG']):
        BGG = LG[key]['BGG']
        target = gtree[ (gtree['hmid']==BGG['id'])&(gtree['timestep']==BGG['timestep']) ]
        tmp = gtree[gtree['last'] == target['last']]
        argsort = np.argsort(-tmp['timestep'])
        rgtree[key] = tmp[argsort]
        ind = isin(nout, rgtree[key]['timestep'])

  0%|          | 0/13 [00:00<?, ?it/s]

In [48]:
keys

[1, 3, 7, 8, 10, 23, 25, 26, 36, 64, 121, 129, 11]

array([1026, 1025, 1024, 1023, 1022, 1021, 1020, 1019, 1018, 1017, 1016,
       1015, 1014, 1013, 1012, 1011, 1010, 1009, 1008, 1007, 1006, 1005,
       1004, 1003, 1002, 1001, 1000,  999,  998,  997,  996,  995,  994,
        993,  992,  991,  990,  989,  988,  987,  986,  985,  984,  983,
        982,  981,  980,  979,  978,  977,  976,  975,  974,  973,  972,
        971,  970,  969,  968,  967,  966,  965,  964,  963,  962,  961,
        960,  959,  958,  957,  956,  955,  954,  953,  952,  951,  950,
        949,  948,  947,  946,  945,  944,  943,  942,  941,  940,  939,
        938,  937,  936,  935,  934,  933,  932,  931,  930,  929,  928,
        927,  926,  925,  924,  923,  922,  921,  920,  919,  918,  917,
        916,  915,  914,  913,  912,  911,  910,  909,  908,  907,  906,
        905,  904,  903,  902,  901,  900,  899,  898,  897,  896,  895,
        894,  893,  892,  891,  890,  889,  888,  887,  886,  885,  884,
        883,  882,  881,  880,  879,  878,  877,  8

In [57]:
tmp = pklload(f"/home/jeon/MissingSat/database/nh2/main_prog/subhalos_00796.pickle")[0]
np.min(tmp['mdm'])

132444214.2078026